In [22]:
import os
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import pytorch_lightning as pl
import matplotlib.pyplot as plt

In [23]:
from neuralforecast import NeuralForecast

ModuleNotFoundError: No module named 'imp'

In [20]:

from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic

ModuleNotFoundError: No module named 'imp'

### 1. Load Data on Money Supply

In [18]:
path = Path(os.getcwd()).parent / "data" / "nbp_money_supply.csv"
data = pd.read_csv(path, sep=";", decimal=",", parse_dates=["yearmon"], date_format="%d.%m.%Y")
data = data.set_index("yearmon")
data

,M0_in_mln_of_PLN,M1_in_mln_of_PLN,M2_in_mln_of_PLN,M3_in_mln_of_PLN
yearmon,,,,
1996-12-01,34196.2,67866.0,140038.7,140428.8
1997-01-01,32391.1,63663.0,138755.5,139333.9
1997-02-01,34594.4,63625.7,141142.5,141692.2
1997-03-01,37713.5,65497.8,144080.8,144931.6
1997-04-01,36533.3,66635.3,147471.8,148012.8
...,...,...,...,...
2023-08-01,457954.3,1598206.5,2182061.0,2192992.0
2023-09-01,432955.8,1622552.6,2225679.6,2233889.8
2023-10-01,464972.6,1621914.9,2230678.0,2241053.7


In [13]:
data = data[data.index >= pd.Timestamp(date(1997, 1, 1))]

In [14]:
data.head()

,M0_in_mln_of_PLN,M1_in_mln_of_PLN,M2_in_mln_of_PLN,M3_in_mln_of_PLN
yearmon,,,,
1997-01-01,32391.1,63663.0,138755.5,139333.9
1997-02-01,34594.4,63625.7,141142.5,141692.2
1997-03-01,37713.5,65497.8,144080.8,144931.6
1997-04-01,36533.3,66635.3,147471.8,148012.8
1997-05-01,39718.0,69033.2,151110.6,151700.6


In [19]:
cutoff_yearmon = pd.Timestamp(2023, 1, 1)

### 2. 

In [ ]:



Y_train_df = AirPassengersPanel[AirPassengersPanel.ds<AirPassengersPanel['ds'].values[-12]] # 132 train
Y_test_df = AirPassengersPanel[AirPassengersPanel.ds>=AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 12 test

model = NBEATSx(h=12, input_size=24,
                #loss=MQLoss(level=[80, 90]),
                loss=DistributionLoss(distribution='Normal', level=[80, 90]),
                scaler_type='robust',
                dropout_prob_theta=0.5,
                stat_exog_list=['airline1'],
                futr_exog_list=['trend'],
                max_steps=200,
                val_check_steps=10,
                early_stop_patience_steps=2)

nf = NeuralForecast(
    models=[model],
    freq='M'
)
nf.fit(df=Y_train_df, static_df=AirPassengersStatic, val_size=12)
Y_hat_df = nf.predict(futr_df=Y_test_df)

# Plot quantile predictions
Y_hat_df = Y_hat_df.reset_index(drop=False).drop(columns=['unique_id','ds'])
plot_df = pd.concat([Y_test_df, Y_hat_df], axis=1)
plot_df = pd.concat([Y_train_df, plot_df])

plot_df = plot_df[plot_df.unique_id=='Airline1'].drop('unique_id', axis=1)
plt.plot(plot_df['ds'], plot_df['y'], c='black', label='True')
plt.plot(plot_df['ds'], plot_df['NBEATSx'], c='purple', label='mean')
plt.plot(plot_df['ds'], plot_df['NBEATSx-median'], c='blue', label='median')
plt.fill_between(x=plot_df['ds'][-12:], 
                 y1=plot_df['NBEATSx-lo-90'][-12:].values, 
                 y2=plot_df['NBEATSx-hi-90'][-12:].values,
                 alpha=0.4, label='level 90')
plt.legend()
plt.grid()
plt.plot()